The **CIS (Cultural Institute/Site and Cultural Event) ontology** aims at modelling the data on cultural institutes or sites such as data regarding the agents that play a specific role on cultural institutes or sites, the sites themselves, the contact points, all multimedia files which describe the cultural institute or site and any other information useful to the public in order to access the institute or site.

We specificially employed it to extract all the cultural events present in the dataset through the property **cis:CulturalEvent**

**cis:CulturalEvent** represents an event that typically is related to the cultural domain

In [7]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_events = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT ?s  WHERE {
 ?s a cis:CulturalEvent.
 }
"""

In [8]:
df = sparql_dataframe.get(endpoint, query_events)

We procede getting the cultural events' names, through using the property, **rdfs:label**. We also kept the IRI in order to have a unique identifier both for institutes and sites that will help us to merge our tables.

In [9]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_event_label = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT ?s ?event ?o ?site ?urlcity ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
}
"""

In [10]:
df = sparql_dataframe.get(endpoint, query_event_label)

We then employed the property **cis:isHostedBySite** to extract all the sites in which the events were held as this specific property links the Event to the Site of the Cultural Institute or Site.

Also in this case we actually extracted both the IRIs and the label through the same property as the one mentioned above.

In [11]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_site= """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT ?s ?event ?o ?site ?urlcity ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
}
"""

In [12]:
df = sparql_dataframe.get(endpoint, query_site)

Then we examined various ways to retrieve the city of each city, examing the various predicates associated to them; the most appropriate and helpful way for us was to first extract the address through **cis:siteAddress** and then from here to get the city through the employment of another ontology (Address (Location) Ontology) and its property **clavpit:hasCity**.

In [13]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_culture_events = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT ?s ?event ?o ?site ?urlcity ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

In [14]:
df = sparql_dataframe.get(endpoint, query_culture_events)

To further clear up this file from any duplicates that might appear we used the pandas method **drop_duplicates** in order to obtain a clearer table of all the data that we obtained together.

In [15]:
df=df.drop_duplicates(["s", "o"])

Finally we saved all of this data in a csv file to then reuse it for our own graph.

In [16]:
df.to_csv("cleancefile.csv")

Then we worked onto counting first the events per sites

In [6]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_site = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT (count (?o) as ?count) ?site WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

In [7]:
df = sparql_dataframe.get(endpoint, query_site)

In [11]:
df.to_csv("numbersite.csv")

Then the events per city.

In [10]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_city = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT (count (?s) as ?count) ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

In [12]:
df = sparql_dataframe.get(endpoint, query_city)

In [13]:
df.to_csv("numbercity.csv")

In [15]:
endpoint = 'https://dati.cultura.gov.it/sparql'

query = '''
SELECT DISTINCT(?urlcity2) ?dbCity WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 owl:sameAs ?dbCity. 
FILTER(contains (str(?dbCity), "dbpedia") )
}
'''

In [16]:
df = sparql_dataframe.get(endpoint, query)

In [17]:
df

,urlcity2,dbCity
0,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Besenello
1,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Calci
2,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Carmignano
3,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Cetona
4,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Chiusi
...,...,...
789,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Casalciprano
790,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Monterosso_Grana
791,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Castrocielo
792,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Lu_e_Cuccaro_Monfe...


In [ ]:
from sparql_dataframe import get
endpoint = 'https://dati.cultura.gov.it/sparql'

#count of all the cities
query = '''
SELECT COUNT(DISTINCT(?urlcity) as ?count) WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
}
'''
CountCity = get(endpoint, query)
CountCity

After extracting 

In [2]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_time="""SELECT DISTINCT ?s ?time ?starttime ?endtime WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s tiapit:atTime ?time.
?time tiapit:startTime ?starttime;
tiapit:endTime ?endtime.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
}"""

In [6]:
df = sparql_dataframe.get(endpoint, query_time)

In [7]:
df

,s,time,starttime,endtime
0,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-09-29 00:00:00,2022-09-29 00:00:00
1,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-09 00:00:00,2022-10-09 00:00:00
2,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-09 00:00:00,2022-12-31 00:00:00
3,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-08 00:00:00,2022-10-09 00:00:00
4,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-09 00:00:00,2022-10-09 00:00:00
...,...,...,...,...
21109,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-12-03 00:00:00,2022-12-03 00:00:00
21110,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-11-30 00:00:00,2022-11-30 00:00:00
21111,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-11-25 00:00:00,2022-11-25 00:00:00
21112,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-11-27 00:00:00,2022-11-27 00:00:00


In [8]:
df.to_csv("time.csv")

In [9]:
query_count_time ="""SELECT DISTINCT (count (?s) as ?count) ?starttime WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s tiapit:atTime ?time.
?time tiapit:startTime ?starttime;
tiapit:endTime ?endtime.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
}"""

In [10]:
df = sparql_dataframe.get(endpoint, query_count_time)

In [11]:
df

,count,starttime
0,3,2022-12-30 00:00:00
1,1,2022-09-19 00:00:00
2,2,2023-02-19 00:00:00
3,3,2023-01-24 00:00:00
4,4,2023-01-30 00:00:00
...,...,...
1258,28,2022-01-27 00:00:00
1259,6,2022-02-05 00:00:00
1260,4,2020-02-03 00:00:00
1261,11,2020-02-21 00:00:00


In [12]:
df.to_csv("time_count.csv")

In [3]:
query_institutions ="""SELECT DISTINCT ?topic WHERE {
?s foaf:primaryTopic ?topic
}"""

df = sparql_dataframe.get(endpoint, query_institutions)

In [4]:
df

,topic
0,http://dati.beniculturali.it/mibact/luoghi/res...
1,http://dati.beniculturali.it/mibact/luoghi/res...
2,http://dati.beniculturali.it/mibact/luoghi/res...
3,http://dati.beniculturali.it/mibact/luoghi/res...
4,http://dati.beniculturali.it/mibact/luoghi/res...
...,...
27560,http://dati.beniculturali.it/mibact/eventi/res...
27561,http://dati.beniculturali.it/mibact/eventi/res...
27562,http://dati.beniculturali.it/mibact/eventi/res...
27563,http://dati.beniculturali.it/mibact/eventi/res...


In [5]:
df.to_csv("institutions.csv")

In [6]:
query_lat_long ="""SELECT DISTINCT ?topic ?lat ?long WHERE {
?s foaf:primaryTopic ?topic.
?topic geo:lat ?lat.
?topic geo:long ?long.
}"""

df = sparql_dataframe.get(endpoint, query_lat_long)

In [7]:
df

,topic,lat,long
0,http://dati.beniculturali.it/mibact/luoghi/res...,46.500280,11.343991
1,http://dati.beniculturali.it/mibact/luoghi/res...,46.073410,11.112929
2,http://dati.beniculturali.it/mibact/luoghi/res...,41.891030,12.487999
3,http://dati.beniculturali.it/mibact/luoghi/res...,42.416588,13.079355
4,http://dati.beniculturali.it/mibact/luoghi/res...,41.561714,14.657649
...,...,...,...
5912,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359
5913,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359
5914,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359
5915,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359


In [8]:
df.to_csv("latlonginstitutions.csv")